# **Template for Torch-Pruning**

This template is just built for your convinience.

You are not required to follow the steps and method given below.

In [1]:
!pip install --upgrade torch_pruning
!pip install torchprofile
# !pip install torch torchvision torchaudio

In [2]:
import torch
import torchvision
from torchvision.models import mobilenet_v2
import torch_pruning as tp
from functools import partial
import copy
import math
import random
import time
from collections import OrderedDict, defaultdict
from typing import Union, List

import numpy as np
from matplotlib import pyplot as plt
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm
import torch.nn.functional as F
from torchprofile import profile_macs
import os

/home/aa35037123/miniconda3/envs/lab2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## A Minimal Example   
In this section, you will perform channel pruning using the library [Torch-Pruning](https://github.com/VainF/Torch-Pruning).  

The puuner in Torch-Pruning has three main functions: sparse training (optional), importance estimation, and parameter removal.  
Torch-pruning offers two core features to support this process:

tp.importance(): This criteria is utilized to measure the importance of weights.  

tp.pruner(): This is a pruner used for the actual pruning of the parameters.  

For detailed information on this process, please refer to this [tutorial](https://github.com/VainF/Torch-Pruning/wiki/4.-High%E2%80%90level-Pruners/). Additionally, a more practical example is available in [here](https://github.com/VainF/Torch-Pruning/blob/master/benchmarks/main.py).

### 1. Load model


In [3]:
model = torch.load('./mobilenetv2_0.963.pth', map_location="cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

### 2. Prepare a pruner
By default, Torch-Pruning will automatically prune the last non-singleton dim of these parameters. If you want to customize this behaviour, please provide an `unwrapped_parameters` list as the following example.

In [4]:
transforms = {
    "train": Compose([
      Resize((224, 224)),
      ToTensor(),
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    "test": Compose([
      Resize((224, 224)),
      ToTensor(),
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

dataset = {}
for split in ["train", "test"]:
  dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transforms[split],
  )

# You can apply your own batch_size
dataloader = {}
for split in ['train', 'test']:
  dataloader[split] = DataLoader(
    dataset[split],
    batch_size=10,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
    drop_last=True
  )

Files already downloaded and verified
Files already downloaded and verified


In [5]:
def progressive_pruning(pruner, model, speed_up, example_inputs, train_loader=None):
    model.eval()
    base_ops, _ = tp.utils.count_ops_and_params(model, example_inputs=example_inputs)
    current_speed_up = 1
    while current_speed_up < speed_up:
#         model.zero_grad()
#         imp=pruner.importance
#         imp._prepare_model(model, pruner)
#         for k, (imgs, lbls) in enumerate(train_loader):
#             if k>=10: break
#             imgs = imgs.cuda()
#             lbls = lbls.cuda()
#             output = model(imgs)
# #             sampled_y = torch.multinomial(torch.nn.functional.softmax(output.cpu().data, dim=1),
# #                                               1).squeeze().cuda()
# #             loss_sample = F.cross_entropy(output, sampled_y)
#             loss_sample = nn.CrossEntropyLoss()(output, lbls)
#             loss_sample.backward()
#             imp.step()
        pruner.step()
        pruned_ops, _ = tp.utils.count_ops_and_params(model, example_inputs=example_inputs)
        current_speed_up = float(base_ops) / pruned_ops
        if pruner.current_step == pruner.iterative_steps:
            break
    return current_speed_up

In [6]:
output_dir = "./pruning_output"
def eval(model, test_loader, device=None, verbose=True):

    num_samples = 0
    num_correct = 0
    
    model.to(device)
    model.eval()
    with torch.no_grad():
        for inputs, targets in tqdm(test_loader, desc="eval", leave=False,
                                    disable=not verbose):
            # Move the data from CPU to GPU
            inputs = inputs.cuda()
            targets = targets.cuda()

            # Inference
            outputs = model(inputs)

            # Convert logits to class indices
            outputs = outputs.argmax(dim=1)

            # Update metrics
            num_samples += targets.size(0)
            num_correct += (outputs == targets).sum()

    return (num_correct / num_samples * 100).item()

In [7]:
model = torch.load('./pruning_output/mobilenet_0.915_61.33.pth', map_location="cpu")
model = model.to(device)
example_inputs = torch.zeros(1, 3, 224, 224).to(device)
ops, size = tp.utils.count_ops_and_params(model, example_inputs=example_inputs)
print("###### choose 1")
MFLOPs = ops/1e6
print(f'MFLOPs: {MFLOPs}')

Accuracy = eval(model, dataloader['test'], device=device) / 100
print(f"Acc: {Accuracy}")

score = max(0, (200-MFLOPs)/(200-45) - 10*min(max(0.92-Accuracy, 0), 1)) * 45
print("Your Score: {:0.2f}/45".format(score))

###### choose 1
MFLOPs: 61.325886


Acc: 0.915199966430664
Your Score: 38.10/45


In [8]:
model = torch.load('./pruning_output/mobilenet_0.906_44.6.pth', map_location="cpu")
model = model.to(device)
example_inputs = torch.zeros(1, 3, 224, 224).to(device)
ops, size = tp.utils.count_ops_and_params(model, example_inputs=example_inputs)
print("##### choose 2")
MFLOPs = ops/1e6
print(f'MFLOPs: {MFLOPs}')

Accuracy = eval(model, dataloader['test'], device=device) / 100
print(f"Acc: {Accuracy}")

score = max(0, (200-MFLOPs)/(200-45) - 10*min(max(0.92-Accuracy, 0), 1)) * 45
print("Your Score: {:0.2f}/45".format(score))

##### choose 2
MFLOPs: 44.622582


Acc: 0.9062000274658203
Your Score: 38.90/45


In [9]:
model = torch.load('./pruning_output/mobilenet_0.908_44.6.pth', map_location="cpu")
model = model.to(device)
example_inputs = torch.zeros(1, 3, 224, 224).to(device)
ops, size = tp.utils.count_ops_and_params(model, example_inputs=example_inputs)
print("##### choose 2")
MFLOPs = ops/1e6
print(f'MFLOPs: {MFLOPs}')

Accuracy = eval(model, dataloader['test'], device=device) / 100
print(f"Acc: {Accuracy}")

score = max(0, (200-MFLOPs)/(200-45) - 10*min(max(0.92-Accuracy, 0), 1)) * 45
print("Your Score: {:0.2f}/45".format(score))

##### choose 2
MFLOPs: 44.622582


Acc: 0.9080999755859375
Your Score: 39.75/45
